# Assignment 1: Preprocessing and Text Classification

Student Name: Chen-An Fan

Student ID: 1087032

### General Info

<b>Due date</b>: Sunday, 28 March 2021 5pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -10% per day (both week and weekend days counted)

<b>Marks</b>: 9% of mark for class (with 8% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See [Using Jupyter Notebook and Python page](https://canvas.lms.unimelb.edu.au/courses/121115/pages/using-jupyter-notebook-and-python?module_item_id=2681264) on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. We recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each question is worth is explicitly given. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

# Overview

In this homework, you'll be working with a collection tweets. The task is to predict the geolocation (country) where the tweet comes from. This homework involves writing code to preprocess data and perform text classification.

# Preprocessing (4 marks)

**Instructions**: Download the data (as1-data.json) from Canvas and put it in the same directory as this iPython notebook. Run the code below to load the json data. This produces two objects, `x` and `y`, which contains a list of  tweets and corresponding country labels (it uses the standard [2 letter country code](https://www.iban.com/country-codes)) respectively. **No implementation is needed.**

In [1]:
import json

x = []
y = []
data = json.load(open("as1-data.json"))
for k, v in data.items():
    x.append(k)
    y.append(v)
    
print("Number of tweets =", len(x))
print("Number of labels =", len(y))
print("\nSamples of data:")
for i in range(10):
    print("Country =", y[i], "\tTweet =", x[i])
    
assert(len(x) == 943)
assert(len(y) == 943)

Number of tweets = 943
Number of labels = 943

Samples of data:
Country = us 	Tweet = @Addictd2Success thx u for following
Country = us 	Tweet = Let's just say, if I were to ever switch teams, Khalesi would be top of the list. #girlcrush
Country = ph 	Tweet = Taemin jonghyun!!! Your birits make me go~ http://t.co/le8z3dntlA
Country = id 	Tweet = depart.senior 👻 rapat perdana (with Nyayu, Anita, and 8 others at Ruang Aescullap FK Unsri Madang) — https://t.co/swRALlNkrQ
Country = ph 	Tweet = Done with internship with this pretty little lady!  (@ Metropolitan Medical Center w/ 3 others) [pic]: http://t.co/1qH61R1t5r
Country = gb 	Tweet = Wow just Boruc's clanger! Haha Sunday League stuff that, Giroud couldn't believe his luck! #clown
Country = my 	Tweet = I'm at Sushi Zanmai (Petaling Jaya, Selangor) w/ 5 others http://t.co/bcNobykZ
Country = us 	Tweet = Mega Fest!!!! Its going down🙏🙌  @BishopJakes
Country = gb 	Tweet = @EllexxxPharrell wow love the pic babe xx
Country = us 	Tweet = You 

### Question 1 (1.0 mark)

**Instructions**: Next we need to preprocess the collected tweets to create a bag-of-words representation. The preprocessing steps required here are: (1) tokenize each tweet into individual word tokens (using NLTK `TweetTokenizer`); (2) lowercase all words; (3) remove any word that does not contain any English alphabets (e.g. {_hello_, _#okay_, _abc123_} would be kept, but not {_123_, _!!_}) and (4) remove stopwords (based on NLTK `stopwords`). An empty tweet (after preprocessing) and its country label should be **excluded** from the output (`x_processed` and `y_processed`).

**Task**: Complete the `preprocess_data(data, labels)` function. The function takes **a list of tweets** and **a corresponding list of country labels** as input, and returns **two lists**. For the first list, each element is a bag-of-words representation of a tweet. For the second list, each element is a corresponding country label. Note that while we do not need to preprocess the country labels (`y`), we need to have a new output list (`y_processed`) because some tweets maybe removed after the preprocessing (due to having an empty set of bag-of-words).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [2]:
'''
1. tokenize each tweet into word tokens
    <Note:> TweetTokenizer keeps hashtags intact
2. lowcase all words
    >> word = word.lower()
3. remove any word that does not contain any English alphabets
4. remove stop words
    must do it after lowcase all words
5. remove empty tweet and its country code
'''

import re
import copy
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from collections import defaultdict

tt = TweetTokenizer()
stopwords = set(stopwords.words('english')) #note: stopwords are all in lowercase

def preprocess_data(data, labels):
    
    ###
    # Your answer BEGINS HERE
    ###
    x_processed = [None] * len(data)
    y_processed = copy.deepcopy(labels)
    emptyTweetIndex = []
    
    for i in range(len(data)): #iterate every tweet
        # tokenize each tweet
        tokenized_tweet = tt.tokenize(data[i])
        # convert each words in a tweet to lower case
        tokenized_tweet[:] = [word.lower() for word in tokenized_tweet]
        
        # in each tweet, remove non-alpha word
        remove_words = [] # The words that wait to be removed
        for word in tokenized_tweet:
            # collect the words that does not contain any English alphabets
            if re.search("[a-zA-Z]+", word) is None:
                remove_words.append(word)   
        for word in remove_words:
            tokenized_tweet.remove(word)
        
        # remove stop words
        tokenized_tweet = [word for word in tokenized_tweet if word not in stopwords]
        x_processed[i] = tokenized_tweet
        # remove empty tweet
        if len(tokenized_tweet) == 0:
            emptyTweetIndex.append(i)
    
    # After pre-processing, remove empty tweet and its corresponding country code    
    for i in sorted(emptyTweetIndex, reverse=True):
        del x_processed[i]
        del y_processed[i]
    
    # convert x_processed from "list of list" to "list of dict (bag-of-word)"
    for tweetIndex in range(len(x_processed)):
        unigramDict = defaultdict(int)
        for word in x_processed[tweetIndex]:
            unigramDict[word] += 1
        x_processed[tweetIndex] = unigramDict
            
    
    
    return x_processed, y_processed
    ###
    # Your answer ENDS HERE
    ###

x_processed, y_processed = preprocess_data(x, y)

print("Number of preprocessed tweets =", len(x_processed))
print("Number of preprocessed labels =", len(y_processed))
print("\nSamples of preprocessed data:")
for i in range(10):
    print("Country =", y_processed[i], "\tTweet =", x_processed[i])

Number of preprocessed tweets = 943
Number of preprocessed labels = 943

Samples of preprocessed data:
Country = us 	Tweet = defaultdict(<class 'int'>, {'@addictd2success': 1, 'thx': 1, 'u': 1, 'following': 1})
Country = us 	Tweet = defaultdict(<class 'int'>, {"let's": 1, 'say': 1, 'ever': 1, 'switch': 1, 'teams': 1, 'khalesi': 1, 'would': 1, 'top': 1, 'list': 1, '#girlcrush': 1})
Country = ph 	Tweet = defaultdict(<class 'int'>, {'taemin': 1, 'jonghyun': 1, 'birits': 1, 'make': 1, 'go': 1, 'http://t.co/le8z3dntla': 1})
Country = id 	Tweet = defaultdict(<class 'int'>, {'depart.senior': 1, 'rapat': 1, 'perdana': 1, 'nyayu': 1, 'anita': 1, 'others': 1, 'ruang': 1, 'aescullap': 1, 'fk': 1, 'unsri': 1, 'madang': 1, 'https://t.co/swrallnkrq': 1})
Country = ph 	Tweet = defaultdict(<class 'int'>, {'done': 1, 'internship': 1, 'pretty': 1, 'little': 1, 'lady': 1, 'metropolitan': 1, 'medical': 1, 'center': 1, 'w': 1, 'others': 1, 'pic': 1, 'http://t.co/1qh61r1t5r': 1})
Country = gb 	Tweet = defau

**For your testing**:

In [3]:
assert(len(x_processed) == len(y_processed))
assert(len(x_processed) > 800)

**Instructions**: Hashtags (i.e. topic tags which start with #) pose an interesting tokenisation problem because they often include multiple words written without spaces or capitalization. Run the code below to collect all unique hashtags in the preprocessed data. **No implementation is needed.**



In [4]:
def get_all_hashtags(data):
    hashtags = set([])
    for d in data:
        for word, frequency in d.items():
            if word.startswith("#") and len(word) > 1:
                hashtags.add(word)
    return hashtags

hashtags = get_all_hashtags(x_processed)
print("Number of hashtags =", len(hashtags))
print(sorted(hashtags))

Number of hashtags = 425
['#100percentpay', '#1stsundayofoctober', '#1yearofalmostisneverenough', '#2011prdctn', '#2015eebritishfilmacademyawards', '#2k16', '#2littlebirds', '#365picture', '#5sosacousticatlanta', '#5sosfam', '#8thannualpubcrawl', '#affsuzukicup', '#aflpowertigers', '#ahimacon14', '#aim20', '#airasia', '#allcity', '#alliswell', '#allwedoiscurls', '#amazing', '#anferneehardaway', '#ariona', '#art', '#arte', '#artwork', '#ashes', '#asian', '#asiangirl', '#askcrawford', '#askherforfback', '#askolly', '#asksteven', '#at', '#australia', '#awesome', '#awesomepict', '#barcelona', '#bart', '#bayofislands', '#beautiful', '#bedimages', '#bell', '#beringmy', '#bettybooppose', '#bff', '#big', '#bigbertha', '#bigbreakfast', '#blackhat', '#blessedmorethanicanimagine', '#blessedsunday', '#blogtourambiente', '#bluemountains', '#bonekachika', '#boomtaob', '#booyaa', '#bored', '#boredom', '#bradersisterhood', '#breaktime', '#breedingground', '#bringithomemy', '#brooksengland', '#burgers'

### Question 2 (1.0 mark)

**Instructions**: Our task here to tokenize the hashtags, by implementing the **MaxMatch algorithm** discussed in class.

NLTK has a list of words that you can use for matching, see starter code below (`words`). Be careful about efficiency with respect to doing word lookups. One extra challenge you have to deal with is that the provided list of words (`words`) includes only lemmas: your MaxMatch algorithm should match inflected forms by converting them into lemmas using the NLTK lemmatizer before matching (provided by the function `lemmatize(word)`). Note that the list of words (`words`) is the only source that you'll use for matching (i.e. you do not need to find  other external word lists). If you are unable to make any longer match, your code should default to matching a single letter.

For example, given "#newrecord", the algorithm should produce: \["#", "new", "record"\].

**Task**: Complete the `tokenize_hashtags(hashtags)` function by implementing the MaxMatch algorithm. The function takes as input **a set of hashtags**, and returns **a dictionary** where key="hashtag" and value="a list of tokenised words".

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [5]:
'''
Tokenize the hashtag by MaxMatch Algorithm.
1. convert the hashtag into lemma
2. match them with the 'words' provided by nltk
'''
from nltk.corpus import wordnet

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
words = set(nltk.corpus.words.words()) #a list of words provided by NLTK
words = set([ word.lower() for word in words ]) #lowercase all the words for better matching
'''
This functions is usde to convert the words in hashtag into lemma
'''
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

'''
output: a dict, key = hashtag, value = list of tokenised words
output the inflected word (the original word)
but when doing the match, use the unflected word
'''
def tokenize_hashtags(hashtags):
    ###
    # Your answer BEGINS HERE
    ###
    tokenized_hashtags = {}
    for hashtag in hashtags:
        tokenized_hashtag = re.split('(#)', hashtag) #split by "#" but keep the separator
        tokenized_hashtag = tokenized_hashtag[1:] #remove the first ''
        assert(len(tokenized_hashtag)==2) #make sure tokenized hashtag is in this form: ['#', 'someStr']
        hashtag_str = tokenized_hashtag[1] #take out the string part
        i = 0
        tokens = []
        while i < len(hashtag_str):
            maxWord = ''
            isMatched = False
            for j in range(i,len(hashtag_str)): #til the end
                tempWord = hashtag_str[i:j+1] #take i~j
                tempWord_lemma = lemmatize(tempWord)
                if (tempWord_lemma in words) and (len(tempWord) > len(maxWord)):
                    maxWord = tempWord
                    isMatched = True #At least match one time
            if isMatched: # At least one match happen
                i = i+len(maxWord)
                tokens.append(maxWord)
            else: # Never match -> jump to the next character
                tokens.append(hashtag_str[i])
                i = i+1
                
        tokenized_hashtag = tokenized_hashtag + tokens
        del tokenized_hashtag[1] # remove the full name of the hashtag
        tokenized_hashtags[hashtag] = tokenized_hashtag
    return tokenized_hashtags
    ###
    # Your answer ENDS HERE
    ###

#tokenise hashtags with MaxMatch
tokenized_hashtags = tokenize_hashtags(hashtags)

#print results
for k, v in sorted(tokenized_hashtags.items())[-30:]:
    print(k, v)

#vanilla ['#', 'vanilla']
#vca ['#', 'v', 'ca']
#vegan ['#', 'vega', 'n']
#veganfood ['#', 'vega', 'n', 'food']
#vegetables ['#', 'vegetables']
#vegetarian ['#', 'vegetarian']
#video ['#', 'video']
#vma ['#', 'v', 'ma']
#voteonedirection ['#', 'vote', 'one', 'direction']
#vsco ['#', 'vs', 'c', 'o']
#vscocam ['#', 'vs', 'coca', 'm']
#walking ['#', 'walking']
#watch ['#', 'watch']
#weare90s ['#', 'wear', 'e', '9', '0', 's']
#wearesocial ['#', 'weares', 'o', 'c', 'i', 'al']
#white ['#', 'white']
#wings ['#', 'wings']
#wok ['#', 'wo', 'k']
#wood ['#', 'wood']
#work ['#', 'work']
#workmates ['#', 'work', 'mates']
#world ['#', 'world']
#worldcup2014 ['#', 'world', 'cup', '2', '0', '1', '4']
#yellow ['#', 'yellow']
#yiamas ['#', 'y', 'i', 'ama', 's']
#ynwa ['#', 'yn', 'wa']
#youtube ['#', 'you', 'tube']
#yummy ['#', 'yummy']
#yws13 ['#', 'y', 'ws', '1', '3']
#zweihandvollfarm ['#', 'z', 'wei', 'hand', 'vol', 'l', 'farm']


**For your testing:**

In [6]:
assert(len(tokenized_hashtags) == len(hashtags))
assert(tokenized_hashtags["#newrecord"] == ["#", "new", "record"])

### Question 3 (1.0 mark)

**Instructions**: Our next task is to tokenize the hashtags again, but this time using a **reversed version of the MaxMatch algorithm**, where matching begins at the end of the hashtag and progresses backwards (e.g. for <i>#helloworld</i>, we would process it right to left, starting from the last character <i>d</i>). Just like before, you should use the provided word list (`words`) for word matching.

**Task**: Complete the `tokenize_hashtags_rev(hashtags)` function by the MaxMatch algorithm. The function takes as input **a set of hashtags**, and returns **a dictionary** where key="hashtag" and value="a list of tokenised words".

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [7]:
def tokenize_hashtags_rev(hashtags):
    ###
    # Your answer BEGINS HERE
    ###
    tokenized_hashtags_rev = {}
    for hashtag in hashtags:
        tokenized_hashtag_rev = re.split('(#)', hashtag)
        tokenized_hashtag_rev = tokenized_hashtag_rev[1:] #remove the first ''
        assert(len(tokenized_hashtag_rev)==2) #make sure tokenized hashtag is in this form: ['#', 'someStr']
        hashtag_str = tokenized_hashtag_rev[1] #take out the string part
        del tokenized_hashtag_rev[-1] #remove the string part, only keep the '#'
        i = len(hashtag_str) - 1
        tokens = []
        while i >= 0:
            maxWord = ''
            isMatched = False
            for j in range(i,-1,-1): # til the index = 0
                tempWord = hashtag_str[j:i+1] #take j~i
                tempWord_lemma = lemmatize(tempWord)
                if (tempWord_lemma in words) and (len(tempWord) > len(maxWord)):
                    maxWord = tempWord
                    isMatched = True #At least match one time
            if isMatched: # At least one match happen
                i = i-len(maxWord)
                tokens[:0] = [maxWord] # insert at the begining (reversed insert)
            else: # Never match -> take out one character
                tokens[:0] = [hashtag_str[i]]
                i = i-1
            
        tokenized_hashtag_rev = tokenized_hashtag_rev + tokens
        # Store in a Disc
        tokenized_hashtags_rev[hashtag] = tokenized_hashtag_rev
    return tokenized_hashtags_rev
    ###
    # Your answer ENDS HERE
    ###

    
#tokenise hashtags with the reversed version of MaxMatch
tokenized_hashtags_rev = tokenize_hashtags_rev(hashtags)

#print results
for k, v in sorted(tokenized_hashtags_rev.items())[-30:]:
    print(k, v)

#vanilla ['#', 'vanilla']
#vca ['#', 'v', 'ca']
#vegan ['#', 'v', 'e', 'gan']
#veganfood ['#', 'v', 'e', 'gan', 'food']
#vegetables ['#', 'vegetables']
#vegetarian ['#', 'vegetarian']
#video ['#', 'video']
#vma ['#', 'v', 'ma']
#voteonedirection ['#', 'vote', 'one', 'direction']
#vsco ['#', 'vs', 'c', 'o']
#vscocam ['#', 'vs', 'c', 'o', 'cam']
#walking ['#', 'walking']
#watch ['#', 'watch']
#weare90s ['#', 'we', 'are', '9', '0', 's']
#wearesocial ['#', 'we', 'are', 'social']
#white ['#', 'white']
#wings ['#', 'wings']
#wok ['#', 'w', 'ok']
#wood ['#', 'wood']
#work ['#', 'work']
#workmates ['#', 'work', 'mates']
#world ['#', 'world']
#worldcup2014 ['#', 'world', 'cup', '2', '0', '1', '4']
#yellow ['#', 'yellow']
#yiamas ['#', 'y', 'i', 'a', 'mas']
#ynwa ['#', 'yn', 'wa']
#youtube ['#', 'you', 'tube']
#yummy ['#', 'yummy']
#yws13 ['#', 'y', 'ws', '1', '3']
#zweihandvollfarm ['#', 'z', 'wei', 'hand', 'vol', 'l', 'farm']


**For your testing:**

In [8]:
assert(len(tokenized_hashtags_rev) == len(hashtags))
assert(tokenized_hashtags_rev["#newrecord"] == ["#", "new", "record"])

### Question 4 (1.0 mark)

**Instructions**: The two versions of MaxMatch will produce different results for some of the hashtags. For a hastag that has different results, our task here is to use a **unigram language model** (lecture 3) to score them to see which is better. Recall that in a unigram language model we compute P(<i>#</i>, <i>hello</i>, <i>world</i> = P(<i>#</i>)\*P(<i>hellow</i>)\*P(<i>world</i>).

You should: (1) use the NLTK's Brown corpus (`brown_words`) for collecting word frequencies (note: the words are already tokenised so no further tokenisation is needed); (2) lowercase all words in the corpus; (3) use add-one smoothing when computing the unigram probabilities; and (4) work in the log space to prevent numerical underflow.

**Task**: Build a unigram language model with add-one smoothing using the word counts from the Brown corpus. Iterate through the hashtags, and for each hashtag where MaxMatch and reversed MaxMatch produce different results, print the following: (1) the hashtag; (2) the results produced by MaxMatch and reversed MaxMatch; and (3) the log probability of each result as given by the unigram language model. Note: you **do not** need to print the hashtags where MaxMatch and reversed MaxMatch produce the same results.

An example output:
```
1. #abcd
MaxMatch = [#, a, bc, d]; LogProb = -2.3
Reversed MaxMatch = [#, a, b, cd]; LogProb = -3.5

2. #efgh
MaxMatch = [#, ef, g, h]; LogProb = -4.2
Reversed MaxMatch = [#, e, fgh]; LogProb = -3.1

```

Have a look at the output, and see if the sequences with better language model scores (i.e. less negative) are generally more coherent.

In [9]:
from nltk.corpus import brown
import math

#words from brown corpus
brown_words = brown.words()

###
# Your answer BEGINS HERE
###
# 1. Find out the different results between "tokenized_hashtags" and "tokenizes_hashtags_rev"
diff_hashtags = []
for key in tokenized_hashtags:
    if tokenized_hashtags[key] != tokenized_hashtags_rev[key]:
        diff_hashtags.append(key)

# 2. Build the uniqram count
#unigram_counts = {}
brown_words_lower = [word.lower() for word in brown_words] #convert to lowercase
word_FD = nltk.FreqDist(brown_words_lower) #use FreqDist as the unigram counter
v=len(word_FD) #v: number of unique word
m=len(brown_words_lower) #m: number of total word
diff_count = 1

for hashtag in diff_hashtags:
    maxMatch = tokenized_hashtags[hashtag] # the result tokens of maxMatch
    maxMatch_rev = tokenized_hashtags_rev[hashtag] # the result tokens of rev maxMatch
    maxMatch_P = [] # the log probability of this tokenization
    maxMatch_rev_P = []
    for w in maxMatch:
        p = math.log((word_FD[w] + 1)/(m+v)) #add-one smoothing
        maxMatch_P.append(p)
    for w in maxMatch_rev:
        p = math.log((word_FD[w] + 1)/(m+v)) #add-one smoothing
        maxMatch_rev_P.append(p)
    print(str(diff_count) + '. ' + hashtag)
    logProb_maxMatch = "{:.2f}".format(sum(maxMatch_P))
    logProb_maxMatch_rev = "{:.2f}".format(sum(maxMatch_rev_P))
    print("Maxmatch = "+str(maxMatch)+"; LogProb = " + logProb_maxMatch)
    print("Reversed Maxmatch = "+str(maxMatch_rev)+"; LogProb = " + logProb_maxMatch_rev +"\n")
    diff_count += 1
del diff_count 


###
# Your answer ENDS HERE
###

1. #vscocam
Maxmatch = ['#', 'vs', 'coca', 'm']; LogProb = -51.59
Reversed Maxmatch = ['#', 'vs', 'c', 'o', 'cam']; LogProb = -59.63

2. #weare90s
Maxmatch = ['#', 'wear', 'e', '9', '0', 's']; LogProb = -66.33
Reversed Maxmatch = ['#', 'we', 'are', '9', '0', 's']; LogProb = -57.36

3. #socal
Maxmatch = ['#', 'soc', 'al']; LogProb = -38.76
Reversed Maxmatch = ['#', 'so', 'cal']; LogProb = -33.73

4. #txlege
Maxmatch = ['#', 't', 'x', 'leg', 'e']; LogProb = -55.11
Reversed Maxmatch = ['#', 't', 'x', 'l', 'e', 'ge']; LogProb = -69.27

5. #innoretail
Maxmatch = ['#', 'inn', 'ore', 'tail']; LogProb = -50.51
Reversed Maxmatch = ['#', 'in', 'no', 'retail']; LogProb = -35.35

6. #skeemsaam
Maxmatch = ['#', 'skee', 'ms', 'aam']; LogProb = -54.93
Reversed Maxmatch = ['#', 's', 'k', 'e', 'ems', 'aam']; LogProb = -74.79

7. #learningcommunties
Maxmatch = ['#', 'learning', 'c', 'om', 'munt', 'ies']; LogProb = -75.13
Reversed Maxmatch = ['#', 'learning', 'c', 'om', 'm', 'unties']; LogProb = -72.29



# Text Classification (4 marks)

### Question 5 (1.0 mark)

**Instructions**: Here we are interested to do text classification, to predict the country of origin of a given tweet. The task here is to create training, development and test partitions from the preprocessed data (`x_processed`) and convert the bag-of-words representation into feature vectors.

**Task**: Create training, development and test partitions with a 70%/15%/15% ratio. Remember to preserve the ratio of the classes for all your partitions. That is, say we have only 2 classes and 70% of instances are labelled class A and 30% of instances are labelled class B, then the instances in training, development and test partitions should also preserve this 7:3 ratio. You may use sklearn's builtin functions for doing data partitioning.

Next, turn the bag-of-words dictionary of each tweet into a feature vector. You may also use sklearn's builtin functions for doing this.

You should produce 6 objects: `x_train`, `x_dev`, `x_test` which contain the input feature vectors, and `y_train`, `y_dev` and `y_test` which contain the labels.

In [10]:
'''
1. create training, development and test partitions from the preprocessed data (x_processed)
    Create training, development and test partitions with a 70%/15%/15% ratio.
2. convert the bag-of-words representation into feature vectors.
    turn the bag-of-words dictionary of each tweet into a feature vector.
'''
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

x_train, x_dev, x_test = None, None, None
y_train, y_dev, y_test = None, None, None

###
# Your answer BEGINS HERE
###
# 1. Create training, development and test partitions with a 70%/15%/15% ratio
# Using stratify to make sure the classes distribution among train, dev, test are the same.
x_train, x_test, y_train, y_test = train_test_split(x_processed, y_processed, test_size=0.30, random_state=33, stratify=y_processed)
x_dev, x_test, y_dev, y_test = train_test_split(x_test, y_test, test_size=0.50, random_state=33, stratify=y_test)
'''
# Examine the classes distribution
a = nltk.FreqDist(y_processed)
b = nltk.FreqDist(y_train)
c = nltk.FreqDist(y_test)
d = nltk.FreqDist(y_dev)
a.tabulate()
b.tabulate()
c.tabulate()
d.tabulate()
'''

# 2. turn the bag-of-words dictionary of each tweet into a feature vector.

vectorizer = DictVectorizer() 
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)
x_dev = vectorizer.transform(x_dev)

# 3. encoded the label
le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)
y_dev = le.fit_transform(y_dev)

###
# Your answer ENDS HERE
###

### Question 6 (1.0 mark)

**Instructions**: Now, let's build some classifiers. Here, we'll be comparing Naive Bayes and Logistic Regression. For each, you need to first find a good value for their main regularisation hyper-parameters, which you should identify using the scikit-learn docs or other resources. Use the development set you created for this tuning process; do **not** use cross-validation in the training set, or involve the test set in any way. You don't need to show all your work, but you do need to print out the **accuracy** with enough different settings to strongly suggest you have found an optimal or near-optimal choice. We should not need to look at your code to interpret the output.

**Task**: Implement two text classifiers: Naive Bayes and Logistic Regression. Tune the hyper-parameters of these classifiers and print the task performance (accuracy) for different hyper-parameter settings.

In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import ParameterGrid
import numpy as np
###
# Your answer BEGINS HERE
###
# Multinomial NB
print("Test different params on Multinominal Naive Bayes")
alpha = [1.0e-10, 0.5, 1.0, 1.5, 2.0, 5] #alpha = laplace smooth; 0 means no smooth.
mNBs = [MultinomialNB(alpha = x) for x in alpha]
for i in range(len(alpha)):
    mNBs[i].fit(x_train, y_train)
    mNB_score = mNBs[i].score(x_dev, y_dev)
    print("Accuracy on dev with alpha="+str(alpha[i])+":   "+str(mNB_score))

# LogisticRegression
print("\nTest different params on Logistic Regission")
loReg_params = ParameterGrid({'solver':['saga'], 'max_iter':[10000],
                'penalty':['l2','l1'], "C":np.logspace(-3,4,8)}) #l1 is more sutiable for large features
# I didn't consider liblinear, because it can not handle multinomial loss
# 'penalty':['none','l1','l2'] don't consider none, because the result is not good
loReg_params2 = ParameterGrid({'solver':['newton-cg', 'lbfgs'], 'max_iter':[10000],
                'penalty':['l2'], "C":np.logspace(-3,4,8),'n_jobs':[-1]})

maxAcc = 0 #max accuracy of logistic regression
bestParams = {}
for params in loReg_params:
    loReg = LogisticRegression(**params)
    loReg.fit(x_train, y_train)
    loReg_score = loReg.score(x_dev, y_dev)
    if loReg_score >= maxAcc:
        maxAcc = loReg_score
        bestParams = params 
    print("Accuracy on dev with "+str(params)+":   "+str(loReg_score))
    print("---")
for params in loReg_params2:
    loReg = LogisticRegression(**params)
    loReg.fit(x_train, y_train)
    loReg_score = loReg.score(x_dev, y_dev)
    if loReg_score >= maxAcc:
        maxAcc = loReg_score
        bestParams = params
    print("Accuracy on dev with "+str(params)+":   "+str(loReg_score))
    print("---")

print("The best Acc of logistic regression is: "+str(maxAcc))
print("The best params of logistic regression is: "+str(bestParams))
    
###
# Your answer ENDS HERE
###

Test different params on Multinominal Naive Bayes
Accuracy on dev with alpha=1e-10:   0.24822695035460993
Accuracy on dev with alpha=0.5:   0.2624113475177305
Accuracy on dev with alpha=1.0:   0.2553191489361702
Accuracy on dev with alpha=1.5:   0.2624113475177305
Accuracy on dev with alpha=2.0:   0.2553191489361702
Accuracy on dev with alpha=5:   0.2624113475177305

Test different params on Logistic Regission
Accuracy on dev with {'C': 0.001, 'max_iter': 10000, 'penalty': 'l2', 'solver': 'saga'}:   0.2624113475177305
---
Accuracy on dev with {'C': 0.001, 'max_iter': 10000, 'penalty': 'l1', 'solver': 'saga'}:   0.10638297872340426
---
Accuracy on dev with {'C': 0.01, 'max_iter': 10000, 'penalty': 'l2', 'solver': 'saga'}:   0.2695035460992908
---
Accuracy on dev with {'C': 0.01, 'max_iter': 10000, 'penalty': 'l1', 'solver': 'saga'}:   0.10638297872340426
---
Accuracy on dev with {'C': 0.1, 'max_iter': 10000, 'penalty': 'l2', 'solver': 'saga'}:   0.2695035460992908
---
Accuracy on dev wi

### Question 7 (1.0 mark)

**Instructions**: Using the best settings you have found, compare the two classifiers based on performance in the test set. Print out both **accuracy** and **macro-averaged F-score** for each classifier. Be sure to label your output. You may use sklearn's inbuilt functions.

**Task**: Compute test performance in terms of accuracy and macro-averaged F-score for both Naive Bayes and Logistic Regression, using their optimal hyper-parameter settings based on their development performance.

In [12]:
###
# Your answer BEGINS HERE
###
#%% Compute test performance in terms of accuracy and macro-averaged F-score for both Naive Bayes and Logistic Regression
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
mNB_best = MultinomialNB(alpha = 0.5)
loReg_best = LogisticRegression(**bestParams)
mNB_best.fit(x_train, y_train)
loReg_best.fit(x_train, y_train)
mNB_predited_y_test = mNB_best.predict(x_test)
loReg_best_predited_y_test = loReg_best.predict(x_test)

mNB_best_acc = accuracy_score(y_test, mNB_predited_y_test)
loReg_best_acc = accuracy_score(y_test, loReg_best_predited_y_test)

mNB_best_macroF1 = f1_score(y_test, mNB_predited_y_test, average='macro')
loReg_best_macroF1 = f1_score(y_test, loReg_best_predited_y_test, average='macro')

print("Accuracy of Multinominal NB on test set is {:.3f}".format(mNB_best_acc))
print("Macro F1 of Multinominal NB on test set is {:.3f}".format(mNB_best_macroF1))

print("Accuracy of Logistic Regression on test set is {:.3f}".format(loReg_best_acc))
print("Macro F1 of Logistic Regression on test set is {:.3f}".format(loReg_best_macroF1))
###
# Your answer ENDS HERE
###

Accuracy of Multinominal NB on test set is 0.317
Macro F1 of Multinominal NB on test set is 0.312
Accuracy of Logistic Regression on test set is 0.366
Macro F1 of Logistic Regression on test set is 0.346


### Question 8 (1.0 mark)

**Instructions**: Print the most important features and their weights for each class for the two classifiers.


**Task**: For each of the classifiers (Logistic Regression and Naive Bayes) you've built in the previous question, print out the top-20 features (words) with the highest weight for each class (countries).

An example output:
```
Classifier = Logistic Regression

Country = au
aaa (0.999) bbb (0.888) ccc (0.777) ...

Country = ca
aaa (0.999) bbb (0.888) ccc (0.777) ...

Classifier = Naive Bayes

Country = au
aaa (-1.0) bbb (-2.0) ccc (-3.0) ...

Country = ca
aaa (-1.0) bbb (-2.0) ccc (-3.0) ...
```

Have a look at the output, and see if you notice any trend/pattern in the words for each country.

In [13]:
###
# Your answer BEGINS HERE
###
import numpy as np

'''
vectorizer.vocabular_ is a dictionary with feature names as key and the encoded features (int) as value.
The encoded features are treated as the column index in the feature weights array.
And the encoded country codes are treated as the row index in the feature weights array.
'''
# This function use the value to find the corresponding key in a dict. 
def getFeatureNameByIndex(vectorizer, ind):
    dic = vectorizer.vocabulary_
    key_list = list(dic.keys())
    val_list = list(dic.values())
    position = val_list.index(ind)
    return key_list[position]

#For Multinominal Naive Bayes, the feature weight is the log prob
mNB_best_fw = mNB_best.feature_log_prob_ #feature weight
print("Classifier = Multinominal Naive Bayes\n")
for label in range(0,mNB_best_fw.shape[0]): #row index is the encoded country codes
    country = le.inverse_transform([label])[0] #decode the country code as a string
    print("Country = {}".format(country))
    ind = np.argsort(mNB_best_fw[label]) #sort the feature weights of this country from small to large, return the index 
    ind = ind[::-1] #inverse the order
    top20ind = ind[:20] #take out the top20 feature weights' index
    for index in top20ind:
        featureName = getFeatureNameByIndex(vectorizer,index)
        featureWeight = mNB_best_fw[label][index]
        print("{} ({:.3f})".format(featureName,featureWeight), end=" ")
    print("\n")

#For Logistic Regression, the feature weight is the coefficient
loReg_best_fw = loReg_best.coef_ #feature weight
print("Classifier = Logistic Regression\n")
for label in range(0,loReg_best_fw.shape[0]):
    country = le.inverse_transform([label])[0] #decode the country code as a string
    print("Country = {}".format(country))
    
    ind = np.argsort(loReg_best_fw[label]) #sort the features from small to large, return the index 
    ind = ind[::-1] #inverse the order
    top20ind = ind[:20]
    for index in top20ind:
        featureName = getFeatureNameByIndex(vectorizer,index)
        featureWeight = loReg_best_fw[label][index]
        print("{} ({:.3f})".format(featureName,featureWeight), end=" ")
    print("\n")        
###
# Your answer ENDS HERE
###

Classifier = Multinominal Naive Bayes

Country = au
i'm (-5.846) australia (-6.013) great (-6.013) one (-6.214) melbourne (-6.465) something (-6.465) good (-6.465) little (-6.465) know (-6.802) vca (-6.802) even (-6.802) tomorrow (-6.802) friends (-6.802) sledging (-6.802) make (-6.802) two (-6.802) shows (-6.802) take (-6.802) lovely (-6.802) go (-6.802) 

Country = ca
i'm (-5.729) one (-6.039) great (-6.039) maybe (-6.240) like (-6.240) thing (-6.240) got (-6.491) life (-6.491) time (-6.491) come (-6.491) first (-6.491) big (-6.491) thanks (-6.491) today (-6.491) u (-6.491) new (-6.491) next (-6.491) school (-6.828) finally (-6.828) sure (-6.828) 

Country = de
i'm (-6.083) love (-6.083) posted (-6.334) night (-6.334) photo (-6.671) markus (-6.671) picture (-6.671) #rosegold (-6.671) year (-6.671) des (-6.671) im (-6.671) tages (-6.671) painting (-6.671) day (-6.671) kernen (-6.671) could (-6.671) remstal (-6.671) https://t.co/dpfrjtdarl (-6.671) medinger (-6.671) photodesign (-6.671

In [14]:
# trend/pattern in the words for each country.

There are some critical patterns to classify the countries.
1. The city name, state name, and the country name. They are most useful to predict the country.
> such as Gauting, Selangor, Remstal, Melbourne, Texas, Chelsea.

2. The users' name
> For example, "pranciska" is a famous name in Indonesia.

3. Special characteristic of country
> Such as "beach" for Australia, "healthy" for Canada, "caterpillar" for Singapore, "pastries" for Philippines.

4. Local language
> such as "yebo" only used in South Africa

Other trends:

It seems that Canadian like think and learn, and they are happy.
Australian like beach, sport, vacation.
Southern Asia countries like to tweet food.
German tweet more art.

